In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
MAX_GENERATIONS = 1000
POPULATION_SIZE = 10
MAX_GRAFT_SIZE = 10
MAX_TREE_SIZE = 35

MAX_GRAD = 1
MAX_ERROR = 1

WEIGHTS_NUM = 3
TEST_POINTS = 100

In [3]:
class Node(dict):
    def __init__(self, *args, **kwds):
        super(Node, self).__init__(*args, **kwds)
        self.__dict__ = self
    
    def __call__(self, x):
        i = [0,0]
        if not isinstance(x, list):
            x = list(x)
        if not isinstance(self.left, int):
            i[0] = x[self.left]
        if not isinstance(self.right, int):
            i[1] = x[self.right]
        return self.op(i, self.weights)
    
    def __getitem__(self, key):
        if key == 0:
            return self.left
        elif key == 1:
            return self.right
        else:
            raise IndexError(f"Tree has only 2 branches, can't get branch {key}")

    def __setitem__(self, key, value):
        if key == 0:
            self.left = value
        elif key == 1:
            self.right = value
        else:
            raise IndexError(f"Tree has only 2 branches,  can't set branch {key}")
    
    def __hash__(self):
        return hash(
            str(hash(self.op))
            +str(hash(self.weights))
            +str(hash(self.left))
            +str(hash(self.right))
        )
    
    def __repr__(self):
        return ' '.join(('( l:', str(self.left), 'r:', str(self.right), ')'))
        


In [4]:
def node_generator(node, test_inputs, test_results):
    i = []
    for point in test_inputs:
        i.append([])
        if isinstance(node.left, int):
            i[-1].append(point[node.left])
        else:
            i[-1].append(node.left(test_inputs))
        if isinstance(node.right, int):
            i[-1].append(point[node.right])
        else:
            i[-1].append(node.right(test_inputs))
    i = tf.constant(i, dtype=tf.float32)
    test_results = tf.constant(test_results, dtype=tf.float32)
    def inner(weights):
        r = tf.TensorArray(tf.float32, size=i.shape[0])
        for p in tf.range(i.shape[0]):
            r = r.write(p, 
            (
                weights[0]*i[p][0]**2 +
                weights[1]*i[p][0] + 
                weights[2]*i[p][0]*i[p][1] +
                weights[3]*i[p][1] +
                weights[4]*i[p][1]**2 +
                weights[5]
            ))
        r = r.stack()
        r = r - test_results
        r = r**2
        return tf.reduce_sum(r)
    return inner

In [63]:
def lse(weights, test_inputs, test_results):
    r = []
    for p, y in zip(test_inputs, test_results):
        r.append((bino(p, weights)-y)**2)
    return sum(r)

In [70]:
import random

MAX_ITERATION = 1000

def gradient_descent_node(node, test_inputs, test_results, descent_rate):
    dw = tf.constant([descent_rate]*6, dtype=tf.float32)
    ELEMENTARY = tf.eye(6, dtype=tf.float32)*dw+tf.ones([6,6])
    grad = 999
    step = 0
    n = node_generator(node, test_inputs, test_results)
    weights = tf.Variable([random.random() for _ in range(6)], dtype=tf.float32)
    while tf.abs(grad) > MAX_GRAD:
        err = tf.TensorArray(size=6, dtype=tf.float32)
        err0 = lse(weights, test_inputs, test_results)
        for i in tf.range(6):
            tf.print(weights*ELEMENTARY[i])
            err = err.write(i, (lse(weights*ELEMENTARY[i], test_inputs, test_results)-err0)/dw[i])
        with tf.GradientTape() as t:
            err2 = n(weights)
        err = err.stack()
        J = t.jacobian(err2, weights)
        tf.print(err, J)
        grad = tf.reduce_sum(err)
        weights.assign_sub(descent_rate*err)
        step += 1
        tf.print(f"step: {step}, J: {err}, grad: {grad}")
        if step > MAX_ITERATION:
            break
    return {"error": err, "grad": grad, "weights": weights}

In [7]:
t_f = lambda x: x[0]**2 + 3*x[0]
t_p = [(random.random()*20-10, random.random()*20-10) for _ in range(500)]
t_y = list(map(t_f,t_p))
t_y = [x/(max(t_y)-min(t_y)) for x in t_y]

In [31]:
bino = lambda x, w: x[0]**2*w[0] + x[0]*w[1] + x[0]*x[1]*w[2] + x[1]**2*w[4] + x[1]*w[3] + w[5]
n = Node(left=0, right=1, op=bino, weights=[0]*6)

In [ ]:
gradient_descent_node(n, t_p, t_y, 1e-6)

[0.574502289 0.715338647 0.683749616 0.232438952 0.450219303 0.511007369]
[0.574501753 0.715339303 0.683749616 0.232438952 0.450219303 0.511007369]
[0.574501753 0.715338647 0.683750272 0.232438952 0.450219303 0.511007369]
[0.574501753 0.715338647 0.683749616 0.232439175 0.450219303 0.511007369]
[0.574501753 0.715338647 0.683749616 0.232438952 0.450219721 0.511007369]
[0.574501753 0.715338647 0.683749616 0.232438952 0.450219303 0.511007845]
[875000 125000 625000 0 750000 0] [1753835.88 38170.2031 721948.062 15447.1416 1595218.12 35626.9102]
step: 1, J: [875000. 125000. 625000.      0. 750000.      0.], grad: 2375000.0
[-0.300498545 0.590338647 0.0587496161 0.232438952 -0.299780697 0.511007369]
[-0.300498247 0.590339184 0.0587496161 0.232438952 -0.299780697 0.511007369]
[-0.300498247 0.590338647 0.058749672 0.232438952 -0.299780697 0.511007369]
[-0.300498247 0.590338647 0.0587496161 0.232439175 -0.299780697 0.511007369]
[-0.300498247 0.590338647 0.0587496161 0.232438952 -0.299781 0.51100

In [12]:
a = tf.eye(6)*1e-7+tf.ones([6,6])

In [13]:
a[0]

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([1.0000001, 1.       , 1.       , 1.       , 1.       , 1.       ],
      dtype=float32)>

In [15]:
weights = tf.Variable([random.random() for _ in range(6)])

In [26]:
import time
n=node_generator(n, t_p, t_y)
t = time.time()
n(weights)
time.time()-t

3.2391796112060547

In [34]:
t = time.time()
lse(n, t_p, t_y)
time.time()-t

0.0015006065368652344

In [51]:
tf.eye(3, dtype=tf.float32)*tf.constant([1,2,3], dtype=tf.float32)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 2., 0.],
       [0., 0., 3.]], dtype=float32)>